In [2]:
import os
import mlflow
import numpy as np
import pandas as pd

from math import sqrt
from joblib import delayed
from joblib import Parallel

from warnings import catch_warnings
from warnings import filterwarnings
from multiprocessing import cpu_count
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.statespace.sarimax import SARIMAX

import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

from pathlib import Path
from functools import reduce
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta

from prophet import Prophet

Importing plotly failed. Interactive plots will not work.


# Data

In [3]:
df = pd.read_csv('./../../final_dfs/for_modeling/df_final_date_wide_2007.csv', converters={'ags2': str, 'ags5': str} )
# fix ags5 missing 0 
df['ags5'] = np.where(df.ags5.str.len() == 4, '0' + df['ags5'], df['ags5'])
df.set_index('ags5', drop=True, inplace=True)
#df = df.drop("Unnamed: 0", axis=1)
df['date'] = pd.to_datetime(df['date'], format = '%Y-%m-%d')
df = df[(df['date'] >= '2007-05-01')]
df

,date,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,number_of_companies_administration,number_of_companies_agriculture,number_of_companies_arts_entertainment,number_of_companies_communication,number_of_companies_construction,number_of_companies_domestic_staff,...,employees_social_security_at_residence,employees_social_security_at_work,realized_short_time_work_companies,realized_short_time_work_people,registerd_jobs,underemployment_without_short_time _work,unemployed,unemployment_benefit_entitled,unemployment_benefit_recipients,unemployment_rate
ags5,,,,,,,,,,,,,,,,,,,,,
01001,2007-05-01,4.0,2.0,21.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.7
01001,2007-06-01,8.0,6.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.2
01001,2007-07-01,9.0,0.0,18.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.5
01001,2007-08-01,6.0,4.0,10.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.3
01001,2007-09-01,11.0,4.0,6.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16073,2021-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0
16074,2021-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6
16075,2021-05-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.8


In [4]:
# select only column for which all values are avaliable
df = df[['date', 
        'number_of_company_deletions', 
        'number_of_company_liquidations', 
        'number_of_start_ups',
        'unemployment_rate', 
        'registerd_jobs',
        'employees_social_security_at_work',
        'employees_social_security_at_residence']]

In [5]:
# rename date and unemployment_rate for Prophet function 
df = df.rename(columns={'date': 'ds', 'unemployment_rate': 'y'})

# 401 Model for each kreis

In [13]:
# one-step forecast
def Prophet_forecast(data, config):
    g, sps, ys, sm, nc, run = config
    # define model
    model = Prophet(growth = g, seasonality_prior_scale = sps, yearly_seasonality = ys, seasonality_mode = sm, 
                   n_changepoints= nc)
    # fit model
    model_fit = model.fit(data)
    # make one step forecast
    future = model_fit.make_future_dataframe(periods=2, freq='MS')
    res = model_fit.predict(future)
    res.to_csv(f"run_res/{g}_{sps}_{ys}_{sm}_{run}.csv")
    return res.iloc[-2:][["ds","yhat"]]

In [7]:
df_ags5 = df.filter(regex='01001', axis=0)
config = ['linear', 15.0, True, 'additive', 80, 3]
Prophet_forecast(df_ags5[(df_ags5['ds'] <= '2019-03-01')], config)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


,ds,yhat
143,2019-04-01,8.029517
144,2019-05-01,7.863308
145,2019-06-01,7.498440


In [8]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
    return sqrt(mean_squared_error(actual, predicted))

In [9]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
    return data[:-n_test], data[-n_test:]

In [10]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
    # split dataset
    train, test = train_test_split(data, n_test)
    # fit model and make forecast for history
    yhat = Prophet_forecast(train, cfg)
    # estimate prediction error
    error = measure_rmse(test['y'], yhat['yhat'])
    return error

In [11]:
def run_model(data, n_test, run): 
        for g in ['linear']: 
            for sps in [10.0]: 
                for ys in [True]: 
                    for sm in ['multiplicative']:
                        for nc in [50]:
                            with mlflow.start_run():
                                params = [g, sps, ys, sm, nc, run]

                                ags5 = list(set(data.index.values.tolist()))
                                rmse = []
                                for i in ags5:
                                    df_ags5 = data.filter(regex=i, axis=0)
                                    current_rmse = walk_forward_validation(df_ags5, n_test, params)
                                    print(current_rmse)
                                    rmse.append(current_rmse)

                                # Log params
                                params = {"growth":g, "sps":sps, "ys":ys, "sm":sm, "nc":nc, "run":run}

                                mlflow.log_params(params)

                                # Log metrics
                                mlflow.log_metric('rmse', np.mean(rmse))
                                mlflow.log_metric('mse', np.mean(rmse)**2)
                                return np.mean(rmse)

In [42]:
run_model(df, 3)

,ds,number_of_company_deletions,number_of_company_liquidations,number_of_start_ups,y,registerd_jobs,employees_social_security_at_work,employees_social_security_at_residence
ags5,,,,,,,,
01001,2007-05-01,4.0,2.0,21.0,12.7,NaN,NaN,NaN
01001,2007-06-01,8.0,6.0,18.0,12.2,NaN,NaN,NaN
01001,2007-07-01,9.0,0.0,18.0,12.5,NaN,NaN,NaN
01001,2007-08-01,6.0,4.0,10.0,12.3,NaN,NaN,NaN
01001,2007-09-01,11.0,4.0,6.0,11.7,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
16077,2020-11-01,3.0,1.0,5.0,6.8,695.0,NaN,NaN
16077,2020-12-01,2.0,1.0,6.0,6.7,702.0,NaN,NaN
16077,2021-01-01,0.0,3.0,2.0,7.3,681.0,NaN,NaN


In [14]:
a1 = run_model(df[(df['ds'] <= '2021-05-01')], 2, 1)
#a2 = run_model(df[(df['ds'] <= '2021-03-01')], 3, 2)
#a3 = run_model(df[(df['ds'] <= '2020-12-01')], 3, 3)
#a4 = run_model(df[(df['ds'] <= '2020-09-01')], 3, 4)
#a5 = run_model(df[(df['ds'] <= '2020-06-01')], 3, 5)
#a6 = run_model(df[(df['ds'] <= '2020-03-01')], 3, 6)
#a7 = run_model(df[(df['ds'] <= '2019-12-01')], 3, 7)
#a8 = run_model(df[(df['ds'] <= '2019-09-01')], 3, 8)
#a9 = run_model(df[(df['ds'] <= '2019-06-01')], 3, 9)
#a10 = run_model(df[(df['ds'] <= '2019-03-01')], 3, 10)


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.284249894956145


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.6680829317159378


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


...

0.1889461902395618


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.32627746309134503


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


0.363537982384384
0.45776232227051367


In [12]:
#df['ds'] + relativedelta(months=+2)


TypeError: unsupported operand type(s) for +: 'DatetimeArray' and 'relativedelta'

In [20]:
np.mean([10.0, 15.0])

12.5

In [9]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
    result = None
    # convert config to a key
    key = str(cfg)
    # show all warnings and fail on exception if debugging
    if debug:
        result = walk_forward_validation(data, n_test, cfg)
    else:
            # one failure during model validation suggests an unstable config
        try:
            # never show warnings when grid searching, too noisy
            with catch_warnings():
                filterwarnings("ignore")
                result = walk_forward_validation(data, n_test, cfg)
        except:
            error = None
    # check for an interesting result
#     if result is not None:
#         print(' > Model[%s] %.3f' % (key, result))
        
    print('.', end='')
    return (key, result)

----------
!mlflow experiments create --experiment-name 401ModelsTemp

Created experiment '401ModelsTemp' with id 6

---------

In [ ]:
# data split
n_test = 12
# model config
cfg = [('linear'), (10.0)]
# train 401 models
print("Training", end=' ')
failed_ags5 = []
rmse_scores = []

with mlflow.start_run(experiment_id=6):
    
    for ags5, df_for_kreis in df_y_grouped:
        
        data = list(df_for_kreis.sort_values('ds').drop(['ds'], axis=1).copy()['y'])
        
        try: 
        
            score = score_model(data, n_test, cfg)

            if score is None:
                raise Exception("No Score")
            
            rmse_scores.append(score[1])
            print('.', end='')
        except:
            failed_ags5.append(ags5)
            print('!', end='')
            
    params = {}
    params['p'], params['d'], params['q'] = cfg[0]
    params['s_p'], params['s_d'], params['s_q'], params['m'] = cfg[1]
    params['t'] = cfg[2]
    
    net_rmse = np.mean([rmse**2 for rmse in rmse_scores])**(0.5)
    
    
    mlflow.log_params(params)
    mlflow.log_metric('net_rmse', net_rmse)
      
    

print('\ndone')
print("Failed ags5:", failed_ags5)

Training ........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [ ]:
rmse_scores

In [ ]:
params

In [37]:
m = Prophet()
m.fit(df)

INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


In [38]:
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
480,2020-11-26
481,2020-11-27
482,2020-11-28
483,2020-11-29
484,2020-11-30


In [2]:
Prophet?